In [1]:
'''
This file will sample the existing 6 GB dataset for a random subset of the
tweets
'''

from util.streamer import line_gen
import pandas as pd
from glob import glob
from itertools import chain
import ujson as json
from util.util import cache
import numpy as np

In [2]:
use_fields = ['created_at', 'in_reply_to_user_id', 'full_text']
user_fields = ['id', 'verified', 'followers_count']


filenames = glob("data/BTC/json/*.jsonl")
files = [open(filename, 'r') for filename in filenames]

jsonl_gen = chain(*files)

# number of posts
n_posts = 100000
# factor of extra comments to include (10x)
comm_mul = 10
posts_lst = []
comm_lst = []

for i, line in enumerate(jsonl_gen):

    line = json.loads(line)

    print(f"\rposts: {len(posts_lst)} comments: {len(comm_lst)}", end='')
    
    row = {}

    for col in use_fields:
        row[col] = line[col]

    for col in user_fields:
        row[col] = line['user'][col]

    # POSTS
    if type(line['in_reply_to_user_id']) != int and not(len(posts_lst) > n_posts):
        posts_lst.append(row)
        # COMMENTS
    elif len(comm_lst) < (n_posts * comm_mul) and line['in_reply_to_user_id'] != None:
        comm_lst.append(row)

        # were done, break
    elif len(comm_lst) > (n_posts * comm_mul):
        break


        
        

posts: 100001 comments: 847743

In [3]:
df = pd.DataFrame(posts_lst + comm_lst)

In [7]:
inter = set(df['id']).intersection(df['in_reply_to_user_id'])

In [10]:
for val in inter:
    print(df[df['id'] == val])

ValueError: ('Lengths must match to compare', (947745,), (34013,))

In [4]:
cache(df, 'btc_data')